In [1]:
import datos_jugadores

# Descargar jugadores de La Liga (2024-2025)

## Obtención de los enlaces de las páginas de todos los jugadores de La Liga

In [2]:
lista_jugadores = datos_jugadores.obtenerEnlacesJugadores()
lista_jugadores

['/jugador/julen-agirrezabala',
 '/jugador/unai-simon',
 '/jugador/de-marcos',
 '/jugador/yuri-b',
 '/jugador/gorosabel',
 '/jugador/vivian',
 '/jugador/i-lekue',
 '/jugador/paredes-2',
 '/jugador/yeray-2',
 '/jugador/vesga',
 '/jugador/ander',
 '/jugador/o-sancet-1',
 '/jugador/r-de-galarreta',
 '/jugador/benat-prados',
 '/jugador/berenguer',
 '/jugador/alvaro-djalo',
 '/jugador/guruzeta',
 '/jugador/williams-jr',
 '/jugador/malcom-adu-ares',
 '/jugador/williams',
 '/jugador/javier-marton',
 '/jugador/nicolas-serrano',
 '/jugador/horatiu-moldovan',
 '/jugador/oblak',
 '/jugador/simeone',
 '/jugador/cesar-azpilicueta',
 '/jugador/reinildo',
 '/jugador/nahuel-molina',
 '/jugador/j-m-gimenez',
 '/jugador/santiago-mourino',
 '/jugador/axel-witsel',
 '/jugador/de-paul',
 '/jugador/arthur-vermeeren',
 '/jugador/pablo-barrios-rivas',
 '/jugador/riquelme',
 '/jugador/koke',
 '/jugador/m-llorente',
 '/jugador/lemar',
 '/jugador/javi-galan',
 '/jugador/samuel-lino',
 '/jugador/griezmann',
 '/ju

## Obtención de los datos de todos los jugadores de La Liga

In [4]:
df_jugadores = datos_jugadores.obtenerDatosJugadores(lista_jugadores)

In [5]:
# Tratamiento de los datos
df_jugadores['fecha'] = df_jugadores.fecha.str.slice(start=0, stop=10)
df_jugadores

,jugador,apodo,fecha,peso,altura,pais,posicion,equipo,dorsal,url
0,Julen Agirrezabala,Agirrezabala,2000-12-26,80,187,ES,Portero,Athletic Club,,https://assets.laliga.com/squad/2024/t174/p470...
1,Unai Simón,Unai Simon,1997-06-11,88,190,ES,Portero,Athletic Club,,https://assets.laliga.com/squad/2024/t174/p212...
2,Óscar De Marcos,De Marcos,1989-04-14,77,182,ES,Defensa,Athletic Club,,https://assets.laliga.com/squad/2024/t174/p590...
3,Yuri Berchiche,Yuri,1990-02-10,79,181,ES,Defensa,Athletic Club,,https://assets.laliga.com/squad/2024/t174/p426...
4,Andoni Gorosabel,Gorosabel,1996-08-04,70,174,ES,Defensa,Athletic Club,,
...,...,...,...,...,...,...,...,...,...,...
345,Santi Comesaña,Santi C.v.,1996-10-05,75,184,ES,Centrocampista,Villarreal Club de Fútbol SAD,,https://assets.laliga.com/squad/2024/t449/p230...
346,Gerard Moreno,Gerard,1992-04-07,77,180,ES,Delantero,Villarreal Club de Fútbol SAD,,https://assets.laliga.com/squad/2024/t449/p937...
347,Alfonso Pedraza,A. Pedraza,1996-04-09,73,184,ES,Delantero,Villarreal Club de Fútbol SAD,,https://assets.laliga.com/squad/2024/t449/p208...
348,Yeremy Pino,Yeremy,2002-10-20,65,172,ES,Delantero,Villarreal Club de Fútbol SAD,,https://assets.laliga.com/squad/2024/t449/p488...


In [6]:
# Almacenamiento de los datos en un Excel
df_jugadores.to_excel('info_jugadores_la_liga.xlsx', index=False)   

# Descargar jugadores de la Premier League (2024-2025)

In [3]:
enlaces_equipos_pl = datos_jugadores.obtenerEnlacesEquiposPremier()
enlaces_equipos_pl

['/clubs/1/Arsenal/squad',
 '/clubs/2/Aston-Villa/squad',
 '/clubs/127/Bournemouth/squad',
 '/clubs/130/Brentford/squad',
 '/clubs/131/Brighton-and-Hove-Albion/squad',
 '/clubs/4/Chelsea/squad',
 '/clubs/6/Crystal-Palace/squad',
 '/clubs/7/Everton/squad',
 '/clubs/34/Fulham/squad',
 '/clubs/8/Ipswich-Town/squad',
 '/clubs/26/Leicester-City/squad',
 '/clubs/10/Liverpool/squad',
 '/clubs/11/Manchester-City/squad',
 '/clubs/12/Manchester-United/squad',
 '/clubs/23/Newcastle-United/squad',
 '/clubs/15/Nottingham-Forest/squad',
 '/clubs/20/Southampton/squad',
 '/clubs/21/Tottenham-Hotspur/squad',
 '/clubs/25/West-Ham-United/squad',
 '/clubs/38/Wolverhampton-Wanderers/squad']

## Obtención de los datos de todos los jugadores de la Premier League

In [5]:
df_jugadores_pl = datos_jugadores.obtenerDatosJugadoresPremier()
df_jugadores_pl

,jugador,equipo,dorsal,posicion,pais,url
0,Aaron Ramsdale,Arsenal,1,Goalkeeper,England,https://resources.premierleague.com/premierlea...
1,Karl Hein,Arsenal,31,Goalkeeper,Estonia,https://resources.premierleague.com/premierlea...
2,David Raya,Arsenal,22,Goalkeeper,Spain,https://resources.premierleague.com/premierlea...
3,Tommy Setford,Arsenal,36,Goalkeeper,England,https://resources.premierleague.com/premierlea...
4,Lucas Martin,Arsenal,54,Goalkeeper,Denmark,https://resources.premierleague.com/premierlea...
...,...,...,...,...,...,...
721,Emilio Ballard-Matthews,Wolverhampton Wanderers,78,Forward,England,https://resources.premierleague.com/premierlea...
722,Leon Chiwome,Wolverhampton Wanderers,84,Forward,England,https://resources.premierleague.com/premierlea...
723,Rodrigo Gomes,Wolverhampton Wanderers,,Forward,Portugal,https://resources.premierleague.com/premierlea...
724,Chiquinho,Wolverhampton Wanderers,20,Forward,Portugal,https://resources.premierleague.com/premierlea...


In [6]:
# Almacenamiento de los datos en un Excel
df_jugadores_pl.to_excel('info_jugadores_premier_league.xlsx', index=False)   

# Descargar jugadores de la Bundesliga (2024-2025)

In [2]:
equipos_bdl = datos_jugadores.obtenerEnlacesEquiposBundesliga()
equipos_bdl

['/de/bundesliga/clubs/fc-augsburg',
 '/de/bundesliga/clubs/1-fc-union-berlin',
 '/de/bundesliga/clubs/vfl-bochum-1848',
 '/de/bundesliga/clubs/sv-werder-bremen',
 '/de/bundesliga/clubs/fc-st-pauli',
 '/de/bundesliga/clubs/borussia-dortmund',
 '/de/bundesliga/clubs/eintracht-frankfurt',
 '/de/bundesliga/clubs/sc-freiburg',
 '/de/bundesliga/clubs/1-fc-heidenheim-1846',
 '/de/bundesliga/clubs/tsg-hoffenheim',
 '/de/bundesliga/clubs/holstein-kiel',
 '/de/bundesliga/clubs/rb-leipzig',
 '/de/bundesliga/clubs/bayer-04-leverkusen',
 '/de/bundesliga/clubs/1-fsv-mainz-05',
 '/de/bundesliga/clubs/borussia-moenchengladbach',
 '/de/bundesliga/clubs/fc-bayern-muenchen',
 '/de/bundesliga/clubs/vfb-stuttgart',
 '/de/bundesliga/clubs/vfl-wolfsburg']

In [3]:
jugadores_bdl = datos_jugadores.obtenerEnlacesJugadoresBundesliga()
jugadores_bdl

['/de/bundesliga/spieler/finn-dahmen',
 '/de/bundesliga/spieler/nediljko-labrovic',
 '/de/bundesliga/spieler/daniel-klein',
 '/de/bundesliga/spieler/robert-gumny',
 '/de/bundesliga/spieler/mads-pedersen',
 '/de/bundesliga/spieler/reece-oxford',
 '/de/bundesliga/spieler/patric-pfeiffer',
 '/de/bundesliga/spieler/jeffrey-gouweleeuw',
 '/de/bundesliga/spieler/dimitrios-giannoulis',
 '/de/bundesliga/spieler/felix-uduokhai',
 '/de/bundesliga/spieler/maximilian-bauer',
 '/de/bundesliga/spieler/keven-schlotterbeck',
 '/de/bundesliga/spieler/noahkai-banks',
 '/de/bundesliga/spieler/david-colina',
 '/de/bundesliga/spieler/yusuf-kabaday',
 '/de/bundesliga/spieler/elvis-rexhbecaj',
 '/de/bundesliga/spieler/arne-maier',
 '/de/bundesliga/spieler/masaya-okugawa',
 '/de/bundesliga/spieler/ruben-vargas',
 '/de/bundesliga/spieler/kristijan-jakic',
 '/de/bundesliga/spieler/tim-breithaupt',
 '/de/bundesliga/spieler/fredrik-jensen',
 '/de/bundesliga/spieler/arne-engels',
 '/de/bundesliga/spieler/niklas-do

In [4]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

In [44]:
def obtenerDatosJugadoresBundesliga():  # Función que devuelve un dataframe con los datos de los jugadores de la Bundesliga
    jugadores_bdl = datos_jugadores.obtenerEnlacesJugadoresBundesliga()
    columnas = {'nombre': [], 'dorsal': [], 'Club': [], 'Position': [], 'Geburtstag': [], 'Nationalität': [], 'Größe': [], 'Gewicht': [], 'foto': []}
    df_jugadores = pd.DataFrame.from_dict(columnas) # Dataframe para almacenar a todos los jugadores
    for i in range(len(jugadores_bdl)):
        lista_columnas = ['nombre', 'dorsal','foto']
        lista_valores = []
        url_jugador = 'https://www.bundesliga.com' + jugadores_bdl[i]
        peticion = requests.get(url_jugador)
        time.sleep(3)
        sopa = BeautifulSoup(peticion.text, 'lxml')
        contenido = sopa.find('header', class_='header container-fluid')
        try:
            nombre = contenido.find('div', class_='firstName').get_text()
        except:
            nombre = ''
        try:
            apellido = contenido.find('div', class_='lastName').get_text()
        except:
            apellido = ''
        nombre = nombre + ' ' + apellido
        lista_valores.append(nombre)
        try:
            dorsal = contenido.find('div', class_='shirtNumber ng-star-inserted').get_text()
        except:
            dorsal = 0
        lista_valores.append(dorsal)
        try:
            imagen = contenido.find('div', class_='playerImage')
            url_imagen = imagen.find('img').get('src')
        except:
            url_imagen = ''
        lista_valores.append(url_imagen)
        etiquetas = contenido.find_all('span', class_='label')
        valores = contenido.find_all('span', class_='value')
        for i in range(len(etiquetas)):
            lista_columnas.append(etiquetas[i].get_text())
        for i in range(len(valores)):
            lista_valores.append(valores[i].get_text())
        nueva_fila = dict(zip(lista_columnas, lista_valores))
        df_jugadores = pd.concat([df_jugadores, pd.DataFrame([nueva_fila])], ignore_index=True)
    df_jugadores = df_jugadores.rename(columns={'Club': 'equipo', 'Position': 'posicion', 'Geburtstag': 'fecha', 'Nationalität': 'pais', 
                                            'Größe': 'altura', 'Gewicht': 'peso'})
    return df_jugadores

## Obtención de los datos de todos los jugadores de la Bundesliga

In [90]:
df_jugadores_bdl = obtenerDatosJugadoresBundesliga()
df_jugadores_bdl

,nombre,dorsal,equipo,posicion,fecha,pais,altura,peso,foto
0,Finn Dahmen,1,FC Augsburg,Torhüter,27.03.1998 (26 Jahre),"Deutsch , Englisch",186 cm,80 kg,https://assets.bundesliga.com/player/dfl-obj-0...
1,Nediljko Labrović,22,FC Augsburg,Torhüter,10.10.1999 (24 Jahre),Kroatisch,196 cm,88 kg,https://assets.bundesliga.com/player/dfl-obj-j...
2,Daniel Klein,25,FC Augsburg,Torhüter,13.03.2001 (23 Jahre),Deutsch,193 cm,87 kg,https://assets.bundesliga.com/player/dfl-obj-0...
3,Robert Gumny,2,FC Augsburg,Verteidigung,04.06.1998 (26 Jahre),Polnisch,182 cm,71 kg,https://assets.bundesliga.com/player/dfl-obj-0...
4,Mads Pedersen,3,FC Augsburg,Verteidigung,01.09.1996 (27 Jahre),Dänisch,174 cm,72 kg,https://assets.bundesliga.com/player/dfl-obj-0...
...,...,...,...,...,...,...,...,...,...
486,Tiago Tomás,11,VfL Wolfsburg,Angriff,16.06.2002 (22 Jahre),Portugiesisch,180 cm,69 kg,https://assets.bundesliga.com/player/dfl-obj-j...
487,Bartosz Bialek,14,VfL Wolfsburg,Angriff,11.11.2001 (22 Jahre),Polnisch,191 cm,84 kg,https://assets.bundesliga.com/player/dfl-obj-0...
488,Kevin Behrens,17,1. FC Union Berlin,Angriff,03.02.1991 (33 Jahre),Deutsch,185 cm,85 kg,https://assets.bundesliga.com/player/dfl-obj-0...
489,Jonas Wind,23,VfL Wolfsburg,Angriff,07.02.1999 (25 Jahre),Dänisch,190 cm,82 kg,https://assets.bundesliga.com/player/dfl-obj-j...


In [116]:
# Tratamiento de los datos
df_jugadores_bdl['fecha'] = (df_jugadores_bdl.fecha.str.slice(start=0, stop=10)).str.replace('.','/')
df_jugadores_bdl['pais'] = [pais[0] for pais in df_jugadores_bdl.pais.str.split(',')]
df_jugadores_bdl['peso'] = [peso[0] for peso in df_jugadores_bdl.peso.str.split(' ')]
df_jugadores_bdl['altura'] = [altura[0] for altura in df_jugadores_bdl.altura.str.split(' ')]
df_jugadores_bdl

,nombre,dorsal,equipo,posicion,fecha,pais,altura,peso,foto
0,Finn Dahmen,1,FC Augsburg,Torhüter,27/03/1998,Deutsch,186,80,https://assets.bundesliga.com/player/dfl-obj-0...
1,Nediljko Labrović,22,FC Augsburg,Torhüter,10/10/1999,Kroatisch,196,88,https://assets.bundesliga.com/player/dfl-obj-j...
2,Daniel Klein,25,FC Augsburg,Torhüter,13/03/2001,Deutsch,193,87,https://assets.bundesliga.com/player/dfl-obj-0...
3,Robert Gumny,2,FC Augsburg,Verteidigung,04/06/1998,Polnisch,182,71,https://assets.bundesliga.com/player/dfl-obj-0...
4,Mads Pedersen,3,FC Augsburg,Verteidigung,01/09/1996,Dänisch,174,72,https://assets.bundesliga.com/player/dfl-obj-0...
...,...,...,...,...,...,...,...,...,...
486,Tiago Tomás,11,VfL Wolfsburg,Angriff,16/06/2002,Portugiesisch,180,69,https://assets.bundesliga.com/player/dfl-obj-j...
487,Bartosz Bialek,14,VfL Wolfsburg,Angriff,11/11/2001,Polnisch,191,84,https://assets.bundesliga.com/player/dfl-obj-0...
488,Kevin Behrens,17,1. FC Union Berlin,Angriff,03/02/1991,Deutsch,185,85,https://assets.bundesliga.com/player/dfl-obj-0...
489,Jonas Wind,23,VfL Wolfsburg,Angriff,07/02/1999,Dänisch,190,82,https://assets.bundesliga.com/player/dfl-obj-j...


In [117]:
# Almacenamiento de los datos en un Excel
df_jugadores_bdl.to_excel('info_jugadores_bundesliga.xlsx', index=False) 